# Modélisation du problème - Projet Green Graph
*Equipe CesiCDP - Chef de projet : Leila | Opérateurs : Tom, Edwin*</br>
## Sommaire :
    1. Introduction
    2. Définition du problème 
        2.1. Description du problème de tournée de livraison
        2.2. Objectifs d’optimisation
    3. Modélisation formelle
        3.1. Hypothèses générales
        3.2. Représentation formelle du réseau
        3.3. Présentation des contraintes supplémentaires retenues
        3.4. Représentation formelle des contraintes choisies
    4. Analyse de la complexité
        4.1. Classification du problème
        4.2. Impact des contraintes supplémentaires sur la complexité
    5. Plan de Travail et Organisation du Projet
        5.1. Étapes prévues 
        5.2. Répartition des tâches dans l’équipe
        5.3. Outils utilisés 
    6. Conclusion
    7. Annexes
        7.1. Glossaire des termes techniques
        7.2. Références bibliographiques

### 1. Introduction

Depuis les années 90, la question environnementale s'est imposé comme une priorité mondiale. Réduction de la consommation d’énergie, diminution des émissions de gaz à effet de serre, développement des mobilités durables… de nombreux objectifs ont été fixés, notamment via des accords internationaux (Protocole de Kyoto, Accords de Paris) et des politiques nationales.

Dans ce cadre, L'ADEME  (Agence de l’Environnement et de la Maîtrise de l’Energie) avec la collaboration de CesiCDP, cherche à optimiser la tournée de ses transports en commun afin de réduire ses coûts financiers et son bilan carbone.

**Comment concevoir une tournée de livraison sur un réseau routier reliant plusieurs points, de manière à minimiser la consommation (temps, distance, carburant, etc.), tout en respectant certaines contraintes opérationnelles (capacités des véhicules, horaires de livraison, etc.) ?**

Ce problème s’apparentant au problème du voyageur de commerce, est connu pour sa complexité algorithmique. Cependant, il est applicable à de nombreux domaines logistiques (livraison de colis, gestion des déchets, interventions technique, etc.).    
Les résultats obtenus pourront être adaptés à différentes tailles de réseau et différents types de territoires (urbain, rural).

L'objectif sera de proposer dans un premier temps une modélisation formelle du problème avec des représentations mathématiques et une analyse de la complexité.   
Par la suite une implémentation des algorithmes avec une démonstration du fonctionnement de celle-ci sur différents cas de test.   
On terminera par une étude expérimentale de nos solutions qui mettra en évidence les perfomances et les limitations de nos algorithmes ainsi qu'une perspective d'amélioration basé sur notre analyse.

### 2. Définition du problème
##### 2.1 Description du problème de tournée de livraison
Dans le cadre de ce projet, on s’intéresse à un problème de logistique urbaine consistant à **planifier une tournée de livraison sur un réseau routier**. L'objectif est de trouver un ou plusieurs itinéraires optimaux pour effectuer des livraisons à un ensemble de points (clients, villes, etc.) à partir d’un point de départ, tout en respectant un ensemble de contraintes (capacités, horaires, accessibilités…).   

Une tournée de livraison telle que nous l'entendons s'apparente au **problème du voyageur de commerce (PVC)** qui consiste à déterminer un itinéraire, le plus court possible, passant par chaque ville une seule fois. 

##### 2.2. Objectifs d’optimisation
L’objectif principal est de minimiser la consommation de ressources, selon plusieurs critères possibles :
| Contraintes | Description |
| ----------- | ----------- |
| Fenêtres temporelles | Chaque point de livraison doit être visité dans un intervalle de temps défini |
| Coût ou restriction sur certaines routes | Certaines routes peuvent être interdites (travaux, fermetures) ou présenter un coût plus élevé (péages, zones réglementées) |
| Dépendances entre visites | Certaines villes doivent être visitées dans un ordre précis |
| Routes dynamiques ou perturbations | Les conditions de circulation peuvent évoluer, modifiant les coûts ou les temps de trajet.  |
| Utilisation de plusieurs véhicules | Le problème peut impliquer plusieurs véhicules réalisant des tournées distinctes, chacun ayant un itinéraire propre |
| Capacités du véhicule | Chaque véhicule a une capacité maximale |
| Équilibrage de la charge des véhicules  | Lorsque plusieurs véhicules sont utilisés, les livraisons doivent être réparties de manière équilibrée pour éviter une surcharge d’un véhicule |

Pour ce fait, notre équipe a pour objectif de mettre en place un algorithme qui prendra en compte **deux contraintes** parmis celles présentées ci-dessus tout en choisissant un chemin le plus court possible.   
Nous prévoyons deux algorithmes de path finding : **l'algorithme A\* et un algorithme génétique**.

### 3. Modélisation formelle
##### 3.1. Hypothèses générales
On pourrais représenter le **réseau routier** par un graphe pondéré, non orienté connexe et complet. C'est à dire qu'on ne prend pas en compte le sens de la circulation et/ *où la pondération serait la distance entre chaque points de livraison*, toutes les villes sont accessibles entre elles donc aucune ville n’est isolée et chaque ville est directement reliée à toutes les autres.    

Les **points de livraison (villes)** sont représentés comme des nœuds/sommets du graphe et les routes par les arêtes.   
Le **dépôt** (point de départ et d’arrivée) est un sommet aléatoire, **un seul véhicule** pars du dépôt, effectue les livraisons puis revient au dépôt.   
Chaque point de livraison doit être visité **une et une seule fois**, *le coût associé à chaque arête correspond à la **distance entre chaque point***. L'objectif est de trouver l'itinéraire avec le coût le plus faible.

##### 3.2. Représentation formelle du réseau
On modélise le réseau routier par un graphe $G = (V,E)$ où :
* $V = \{v0,v1,v2,...,vn\}$ est l’ensemble des sommets représentant le dépôt et les points de livraison
* on note le sommet du dépot, $v0$
* $E$ est l’ensemble des arêtes représentant les routes entre les sommets, chaque arête du graphe relie deux sommets de $V$, il contient toutes les paires $(u, v)$ représentant les connexions (routes) dans le graphe.

##### 3.3. Présentation des contraintes supplémentaires retenues
Dans notre cas, nous avons choisi deux contraintes supplémentaires qui rendent la modélisation plus réaliste :
* ***Coût ou restriction de passage sur certaines arêtes :** Certaines routes peuvent être plus coûteuses ou interdites (par exemple, travaux ou routes bloquées).*
* **Dépendances entre visites :** Une ville ne peut être visitée qu'après en avoir visité une autre (par exemple, une livraison doit précéder une collecte).   

##### 3.4. Représentation formelle des contraintes choisies

### 4. Analyse de la complexité

##### 4.1. Classification du problème
##### **Démonstration de la NP-Complétude du Problème du Voyageur de Commerce (PVC)**

##### **Construction de l'instance de PVC**
*inserer une image représentant les graphes*    
Soit $(I_{CH})$ une instance du Cycle Hamiltonien sur un graphe $(G = (V, E))$. Nous construisons une instance $(I_{PVC})$ de PVC comme suit :
1. **Graphe $(G' = (V, E'))$** : 
   - $(E')$ contient toutes les arêtes de $(G)$ avec un coût de **1**.
   - Les arêtes absentes de $(G)$ (i.e., celles de son complément $(\overline{G})$) sont ajoutées à $(E')$ avec un coût de **2**.
2. **Valeur $(k = |V|)$** : La longueur maximale autorisée pour le cycle PVC est fixée à $(|V|)$.

Cette construction s'effectue en temps polynomial (en $(O(|V|^2))$).

##### **Réduction et Preuve d'Équivalence**
1. **Si $(G)$ a un Cycle Hamiltonien** :
   - Ce cycle utilise exactement $(|V|)$ arêtes de $(G)$, toutes de coût **1**.
   - Dans $(I_{PVC})$, ce cycle correspond à un circuit de coût total $(|V|)$, répondant "oui" à $(I_{PVC})$.

2. **Si $(I_{PVC})$ répond "oui"** :
   - Le circuit PVC a un coût $(\leq |V|)$. Comme les arêtes de $(\overline{G})$ coûtent **2**, le circuit ne peut en utiliser aucune (sinon le coût total dépasserait $(|V|)$).
   - Le circuit utilise donc uniquement des arêtes de $(G)$, formant un Cycle Hamiltonien dans $(G)$.

3. **Si $(G)$ n'a pas de Cycle Hamiltonien** :
   - Tout circuit PVC dans $(G')$ doit utiliser au moins une arête de $(\overline{G})$, ce qui entraîne un coût $(\geq |V| + 1)$.
   - $(I_{PVC})$ répond donc "non".


##### **Conclusion**
**PVC est NP-Difficile** : Toute instance du PVC (NP-Complet) se réduit polynomialement au cycle hamiltonien, de fait il est au moins aussi difficile que le problème du cycle hamiltonien, conjugué au PVC il devient donc NP-Complet (NP + NP-Difficile).

##### 4.2. Impact des contraintes supplémentaires sur la complexité
*WIP*

### 5. Plan de Travail et Organisation du Projet
##### 5.1. Étapes prévues 
| Date de livraison | Description | Etat |
| ----------- | ----------- | - |
| **09/04/2025 - 15/04/2025** | **Livrable 1** | ❌ |
| 09/04/2025 - 09/04/2025 | Reformulation de la problématique | ✅ |
| 09/04/2025 - 12/04/2025 | Calcul de complexité | ❌ |
| 10/04/2025 - 14/04/2025 | Représentation formelle des données | ✅ |
| 10/04/2025 - 12/04/2025 | Représentation formelle des problèmes | ❌ |
| 13/04/2025 - 15/04/2025 | Représentation formelle des objectifs à optimiser | ❌ |
| **15/04/2025 - 28/04/2025** | **Livrable 2** | ❌ |
| 15/04/2025 - 28/04/2025 | Mise à jour de la modélisation du livrable 1  | ❌ |
| 15/04/2025 - 16/04/2025 | Décrit les méthodes de résolution choisies : détails sur les algorithmes utilisés | ❌ |
| 15/04/2025 - 17/04/2025 | L’implémentation de ces algorithmes | ❌ |
| 16/04/2025 - 18/04/2025 | Etude expérimentale : Plan d'expérience | ❌ |
| 18/04/2025 - 19/04/2025 | Etude expérimentale : benchmarks | ❌ |
| 20/04/2025 - 24/04/2025 | Etude expérimentale : limitations / améliorations possibles | ❌ |

##### 5.2. Outils utilisés 
- VSCode
- Jupyter notebook
- Git/Github
- Office
- pandoc

**Langages et librairies :**
- Python 3.12
- Package de lib pour OpenStreetMap
- matplotlib
- numpy
- ipywidgets

### 6. Conclusion

### 7. Annexes
##### 7.1. Glossaire des termes techniques
- **Chemin Hamiltonien** : Un chemin hamiltonien est un chemin passant par tous les sommets d'un graphe sans repasser sur les arêtes.
- **Cycle Hamiltonien** : Un cycle hamiltonien est un chemin hamiltonien faisant un cycle.
- **Graphe pondéré** : Un graphe pondéré possède une pondération sur ses arêtes permettant de statuer de leur valeur.
- **Graphe connexe** : un graphe est connexe lorsque tous ses sommets sont reliés par des arêtes et forment un seul corps. 
- **Graphe complet** : Un graphe complet possède des arêtes pour chacun de ses sommets, les reliants à tous les autres sommets du graphe.
- **Degré** : le degré d'un sommet correspond au nombre d'arêtes connectées à celui-ci.
- **Ordre** : L'ordre d'un graphe est son nombre de sommets.
- **NP, NP-Difficile, NP-Complet** : En informatique théorique, et plus précisément en théorie de la complexité, une classe de complexité est un ensemble de problèmes algorithmiques dont la résolution nécessite la même quantité d'une certaine ressource.
##### 7.2. Références bibliographiques

### 8. Données de Test
Pour répondre à ce problème nous utiliserons les villes de france obtenu grâce à World City DB. 

In [2]:
import requests
import json
from geopy.geocoders import Nominatim

def geocode_city(city_name):
    """Converts a city name to coordinates (latitude, longitude)"""
    geolocator = Nominatim(user_agent="routing_app")
    location = geolocator.geocode(city_name)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        raise ValueError(f"Unable to find coordinates for {city_name}")

def calculate_travel_time(departure_city, arrival_city, mode, osrm_link, params):
    """
    Calculates travel time between two cities using the OSRM API
    
    Args:
        departure_city (str): Name of the departure city
        arrival_city (str): Name of the arrival city
        mode (str): Transportation mode (driving, cycling, walking)
        
    Returns:
        tuple: (time in seconds, distance in meters, formatted time)
    """
    # Get city coordinates
    try:
        lat1, lon1 = geocode_city(departure_city)
        lat2, lon2 = geocode_city(arrival_city)
    except ValueError as e:
        return (None, None, str(e))
    
    # Building the URL for the OSRM API
    url = f"{osrm_link}/{mode}/{lon1},{lat1};{lon2},{lat2}"

    # Call to the OSRM API
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return (None, None, f"Error during API call: {response.status_code}")
    
    data = response.json()
    
    if data["code"] != "Ok":
        return (None, None, f"OSRM API error: {data['code']}")
    
    # Extracting time and distance information
    route = data["routes"][0]
    duration_seconds = route["duration"]
    distance_meters = route["distance"]
    
    # Formatting time for display
    hours, remainder = divmod(duration_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    formatted_time = ""
    if hours > 0:
        formatted_time += f"{int(hours)} hour{'s' if hours > 1 else ''} "
    if minutes > 0:
        formatted_time += f"{int(minutes)} minute{'s' if minutes > 1 else ''}"
    
    return (duration_seconds, distance_meters, formatted_time.strip())

def display_route(departure_city, arrival_city, mode, osrm_link, params):
    """Displays route information between two cities"""
    modes = {
        "driving": "by car",
        "cycling": "by bicycle",
        "walking": "on foot"
    }
    
    duration, distance, message = calculate_travel_time(departure_city, arrival_city, mode, osrm_link, params)
    
    if duration is None:
        print(message)
        return
    
    print(f"Route from {departure_city} to {arrival_city} {modes.get(mode, '')}:")
    print(f"Travel time: {message}")
    print(f"Distance: {distance/1000:.1f} km")



ModuleNotFoundError: No module named 'requests'

In [ ]:
print(display_route("Reims", "Paris", "driving", "https://osrm.panini.simon511000.fr/route/v1", {
        "overview": "false",
        "alternatives": "false",
        "toll" : "false",
    }))

In [ ]:
### Matrix generation ###
from tabulate import tabulate

def matrix_generation(cities, mode, link, params):
    matrix = []

    for sourceCity in cities:
        submatrix = []
        for destinationCity in cities:
            if sourceCity is not destinationCity:
                duration, distance, _ = calculate_travel_time(sourceCity, destinationCity, mode, link, params)
                submatrix.append([duration,distance])
            
            else:
                submatrix.append([0,0])
        matrix.append(submatrix)

    return matrix

Cities = ["reims", "lille", "paris", "bordeaux"]
link = "http://router.project-osrm.org/route/v1"
params = {
    "overview": "false",
    "alternatives": "false",
    }

print(tabulate(matrix_generation(Cities, "driving", link, params), headers=Cities, showindex=Cities, tablefmt="fancy_grid") )

╒══════════╤═════════════════════╤═════════════════════╤═════════════════════╤═════════════════════╕
│          │ reims               │ lille               │ paris               │ bordeaux            │
╞══════════╪═════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╡
│ reims    │ [0, 0]              │ [7639.3, 199255.5]  │ [6042.2, 144045.8]  │ [26773.1, 718258.2] │
├──────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ lille    │ [7840, 199995]      │ [0, 0]              │ [9323.2, 221833.6]  │ [30241.9, 805329]   │
├──────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ paris    │ [6027.6, 143738.5]  │ [9077.3, 221616.7]  │ [0, 0]              │ [22062.7, 583843.9] │
├──────────┼─────────────────────┼─────────────────────┼─────────────────────┼─────────────────────┤
│ bordeaux │ [26671.7, 717632.2] │ [29972.6, 805252.5] │ [21983.1, 582172.2] │ [0, 0]      